In [0]:
SCALE =  1.0                   # scale for k e.g. 1.0
FOLDER = "269_svd_d_optimal_1" # folder name on drive e.g. "269_svd_d_optimal_5"

In [2]:
!pip install torch torchvision

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision

from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torchvision.utils as vutils


import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np


dtype = torch.FloatTensor

if torch.cuda.is_available():
  dtype = torch.cuda.FloatTensor
  
print(dtype)

<class 'torch.cuda.FloatTensor'>


In [3]:
image_size = 32
batch_size = 125
transform = transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))   #Normalizes to [-1, 1]
                           ])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

# testset = torchvision.datasets.CIFAR10(root='./data', train=False,
#                                        download=True, transform=transform)
# testloader = torch.utils.data.DataLoader(testset, batch_size=1000,
#                                          shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

0it [00:00, ?it/s]

100%|█████████▉| 170287104/170498071 [01:10<00:00, 3643247.74it/s]

In [4]:
from google.colab import drive
%cd ..
!rm -rf GAN-Stabilization/
%cd -

!git clone https://github.com/thearnavgarg/GAN-Stabilization.git
%cd GAN-Stabilization

/
/content
Cloning into 'GAN-Stabilization'...
remote: Enumerating objects: 116, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 116 (delta 60), reused 78 (delta 28), pack-reused 0
Receiving objects: 100% (116/116), 41.05 MiB | 31.35 MiB/s, done.
Resolving deltas: 100% (60/60), done.
/content/GAN-Stabilization


In [5]:
from layers import SVDConv2d
from gan import Generator, Discriminator, weights_init

#Connect google drive for loading and saving weights
from google.colab import drive
drive.mount('/content/drive')

170500096it [01:30, 3643247.74it/s]                               

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
nz = 100             # Number of latent z variables
nc = 3               # Number of channels
ngpu = 1
# Generator
ngf = 64
netG = Generator(ngpu, nz, ngf, nc).type(dtype)
netG.apply(weights_init)
print(netG)

# Discrimiator
ndf = 64
netD = Discriminator(ngpu, nz, ndf, nc, SCALE).type(dtype)
print(netD)

In [0]:
criterion = nn.BCELoss().type(dtype)
beta1 = 0.5
lr = 0.0002
niter = 250

fixed_noise = torch.randn(64, nz, 1, 1).type(dtype)
real_label = 1
fake_label = 0

# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [0]:
import time
lbda = 0.1
gamma = 10
noise = torch.randn(batch_size, nz, 1, 1).type(dtype)
img_list = []
start = time.time()
print("starting training at: " + str(start))
prev = time.time()
with open('/content/drive/My Drive/' + FOLDER +'/log.txt', 'w') as f:
    for epoch in range(niter):
        curr = time.time()
        print("*"*10)
        print("Time for last epoch: " + str(curr-prev))
        f.write("*"*10 + "\n")
        f.write("Time for last epoch: " + str(curr-prev) + "\n")
        prev = curr
        for i, data in enumerate(trainloader, 0):
            ############################
            # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
            ###########################
            # train with real
            netD.zero_grad()
            real_cpu = data[0].type(dtype)
            label = torch.full((batch_size,), real_label).type(dtype)

            output = netD(real_cpu).view(-1)
            errD_real = criterion(output, label)
            errD_real += lbda*netD.orth_reg()
            errD_real += gamma*netD.D_optimal_reg()
            errD_real.backward()
            D_x = output.mean().item()

            # train with fake
            noise = torch.randn(batch_size, nz, 1, 1).type(dtype)
            fake = netG(noise)
            label.fill_(fake_label)
            output = netD(fake.detach()).view(-1)
            errD_fake = criterion(output, label)
            errD_fake.backward()

            D_G_z1 = output.mean().item()
            errD = errD_real + errD_fake
            optimizerD.step()

            ############################
            # (2) Update G network: maximize log(D(G(z)))
            ###########################
            netG.zero_grad()
            label.fill_(real_label)  # fake labels are real for generator cost
            output = netD.forward(fake).view(-1)
            errG = criterion(output, label)
            errG.backward()
            D_G_z2 = output.mean().item()
            optimizerG.step()

            if i % 50 == 0:
                print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
                      % (epoch, niter, i, len(trainloader),
                         errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
                f.write('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f\n'
                      % (epoch, niter, i, len(trainloader),
                         errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        torch.save(netG.state_dict(), '/content/drive/My Drive/' + FOLDER +'/G.pt')
        torch.save(netD.state_dict(), '/content/drive/My Drive/' + FOLDER +'/D.pt')
        torch.save(optimizerD.state_dict(), '/content/drive/My Drive/' + FOLDER +'/G_optim.pt')
        torch.save(optimizerG.state_dict(), '/content/drive/My Drive/' + FOLDER +'/D_optim.pt')

        if epoch % 5 == 0:
            #Visualization of images
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))
            fake_images = np.transpose(img_list[-1],(1,2,0))
            plt.imsave('/content/drive/My Drive/' + FOLDER +'/fake_image{}.png'.format(epoch), fake_images)
    f.close()

In [0]:
with torch.no_grad():
    fake = netG(fixed_noise).detach().cpu()
img_list.append(vutils.make_grid(fake, padding=2, normalize=True))
fake_images = np.transpose(img_list[-1],(1,2,0))
plt.imsave('/content/drive/My Drive/' + FOLDER +'/fake_image{}.png'.format(epoch), fake_images)